<H1> Language model</H1>

In [1]:
# /root will be the main dir
H = %env HOME
%cd $H

/root


In [0]:
#kenlm dependencies
!apt update && apt install -y --no-install-recommends \
 build-essential cmake \
 libboost-system-dev libboost-thread-dev libboost-program-options-dev libboost-test-dev \
 libeigen3-dev zlib1g-dev libbz2-dev liblzma-dev
#kenlm
!git clone https://github.com/kpu/kenlm && \
	cd kenlm && \
	git checkout 87e85e66c99ceff1fab2500a7c60c01da7315eec && \
	mkdir -p build && \
	cd build && \
	cmake .. && \
	make -j 4


In [0]:
%cd $H
!git clone https://github.com/mozilla/DeepSpeech.git
# 0.8.0 release
!(cd DeepSpeech && git checkout f56b07dab4542eecfb72e059079db6c2603cc0ee)
!git clone https://github.com/MozillaItalia/DeepSpeech-Italian-Model.git

In [0]:
# Run this cell if you need the mitads.txt file
!apt install pixz
# TODO tr upper to lower is useless?
!curl -sSL https://github.com/MozillaItalia/DeepSpeech-Italian-Model/releases/download/Mitads-1.0.0-alpha/mitads-1.0.0-alpha.tar.xz --output mitads.tar.xz
!pixz -d mitads.tar.xz mitads.tar
!tar -xvf mitads.tar

In [ ]:
%cd DeepSpeech/data/lm
!set -xe
!python generate_lm.py --input_txt "$H/mitads.txt" --output_dir "$H" \
  --top_k 500000 --kenlm_bins "/root/kenlm/build/bin" \
  --arpa_order 5 --max_arpa_memory "75%" --arpa_prune "0|0|1" \
  --binary_a_bits 255 --binary_q_bits 8 --binary_type trie

In [16]:
%cd $H/DeepSpeech/data/lm
!curl -LO "https://github.com/mozilla/DeepSpeech/releases/download/v0.8.0/native_client.amd64.cpu.linux.tar.xz"
!tar xvf native_client.*.tar.xz
!./generate_scorer_package --alphabet $H/DeepSpeech-Italian-Model/DeepSpeech/italian_alphabet.txt \
  --lm "$H/lm.binary" \
  --vocab "$H/vocab-500000.txt" \
  --package "$H/kenlm.scorer" \
  --default_alpha 0.931289039105002 \
  --default_beta 1.1834137581510284


/root/DeepSpeech/data/lm
500000 unique words read from vocabulary file.
Doesn't look like a character based model.
Using detected UTF-8 mode: False
Package created in /root/kenlm.scorer
